In [ ]:



print("Uyumlu kütüphane versiyonları kuruluyor... Lütfen bekleyin.")
%pip install --upgrade --quiet "langchain==0.1.20" "langchain-community==0.0.38" "langchain-core==0.1.52" "langchain-google-genai" "langchain-chroma" "langchain-huggingface" "torch" "sentence-transformers" "beautifulsoup4" "jq" "numpy<2.0.0"

import os
import json
from langchain_community.document_loaders import JSONLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma
from langchain.chains import ConversationalRetrievalChain
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.memory import ConversationBufferWindowMemory
from langchain_core.prompts import PromptTemplate

print("\n" + "="*50)
print("Kurulum tamamlandı. Ortam başarıyla yüklendi.")


In [ ]:

import json

project_root = r"C:\Users\yedis\Desktop\yehu\Github Repo\MentorMate-SSS"
output_path = os.path.join(project_root, "output")
enriched_file_path = os.path.join(output_path, "enriched_dataset.jsonl")
generated_file_path = os.path.join(output_path, "generated_data_google.jsonl")


print("Veri setleri temel formatta yükleniyor...")
loader_enriched = JSONLoader(
    file_path=enriched_file_path,
    jq_schema='.',
    json_lines=True,
    text_content=False
)
loader_generated = JSONLoader(
    file_path=generated_file_path,
    jq_schema='.',
    json_lines=True,
    text_content=False
)
documents_enriched = loader_enriched.load()
documents_generated = loader_generated.load()
all_documents_raw = documents_enriched + documents_generated
print(f"Başarıyla yüklendi. {len(all_documents_raw)} ham döküman bulundu.")

print("Dökümanlar 'Soru-Cevap' formatına dönüştürülüyor...")
all_documents = []
for doc in all_documents_raw:
    try:

        record = json.loads(doc.page_content) if isinstance(doc.page_content, str) else doc.page_content
        
        question = record.get("question") or record.get("soru", "Soru belirtilmemiş")
        answer = record.get("answer") or record.get("cevap", "Cevap belirtilmemiş")
        source = record.get("source", "N/A")

        new_content = f"Soru: {question}\nCevap: {answer}"
        
        doc.page_content = new_content
        doc.metadata = {"source": source}
        all_documents.append(doc)
        
    except Exception as e:
        print(f"Uyarı: Bir satır işlenirken hata oluştu ve atlandı: {e}")
        continue

print("Dönüştürme işlemi tamamlandı!")
print(f"Toplam döküman sayısı: {len(all_documents)}")


In [ ]:

import json
import os


file_to_check = r"C:\Users\yedis\Desktop\yehu\Github Repo\MentorMate-SSS\output\enriched_dataset.jsonl"

print(f"'{os.path.basename(file_to_check)}' dosyası kontrol ediliyor...")

error_found = False
try:
    with open(file_to_check, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            try:
                json.loads(line)
            except json.JSONDecodeError as e:
                print(f"\n!!! HATA BULUNDU !!!")
                print(f"Sorunlu Satır Numarası: {i + 1}")
                print(f"Hata Mesajı: {e}")
                print("\n--- Sorunlu Satırın İçeriği ---")
                print(line)
                print("---------------------------------")
                error_found = True
                break
    
    if not error_found:
        print("\nDosya kontrol edildi, herhangi bir JSON format hatası bulunamadı.")

except FileNotFoundError:
    print(f"HATA: Dosya bulunamadı! -> {file_to_check}")
except Exception as e:
    print(f"Beklenmedik bir hata oluştu: {e}")

In [ ]:

import json
import os

file_to_check = r"C:\Users\yedis\Desktop\yehu\Github Repo\MentorMate-SSS\output\generated_data_google.jsonl"

print(f"'{os.path.basename(file_to_check)}' dosyası kontrol ediliyor...")

error_found = False
try:
    with open(file_to_check, 'r', encoding='utf-8') as f:
        for i, line in enumerate(f):
            if not line.strip():
                print(f"\n!!! UYARI: Boş satır bulundu !!!")
                print(f"Satır Numarası: {i + 1}")
                error_found = True
                break

            try:
                data = json.loads(line)
                if 'question' not in data:
                    print(f"\n!!! HATA BULUNDU: 'soru' anahtarı eksik !!!")
                    print(f"Sorunlu Satır Numarası: {i + 1}")
                    print("\n--- Sorunlu Satırın İçeriği ---")
                    print(line)
                    print("---------------------------------")
                    error_found = True
                    break 
            except json.JSONDecodeError as e:
                
                print(f"\n!!! HATA BULUNDU: JSON format hatası !!!")
                print(f"Sorunlu Satır Numarası: {i + 1}")
                print(f"Hata Mesajı: {e}")
                print("\n--- Sorunlu Satırın İçeriği ---")
                print(line)
                error_found = True
                break

    if not error_found:
        print("\nDosya kontrol edildi, 'soru' anahtarı tüm satırlarda mevcut.")

except FileNotFoundError:
    print(f"HATA: Dosya bulunamadı! -> {file_to_check}")
except Exception as e:
    print(f"Beklenmedik bir hata oluştu: {e}")

In [ ]:

import os
import json
import re
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.docstore.document import Document
import shutil 


def clean_text(text):
    text = re.sub(r'\*\*', '', text) 
    text = text.strip()              
    return text

project_root = ".." 
db_path = os.path.join(project_root, "chroma_db")
collection_name = "mentormate_faq"

file_paths = [
    os.path.join(project_root, "output", "generated_data_google.jsonl"),
    os.path.join(project_root, "output", "enriched_dataset.jsonl")
]


if os.path.exists(db_path):
    print(f"'{db_path}' konumunda mevcut bir veritabanı bulundu. Temiz kurulum için siliniyor...")
    shutil.rmtree(db_path) 
    print("Eski veritabanı silindi.")

print("Veritabanı sıfırdan oluşturulacak...")

print("Embedding modeli yükleniyor...")
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={'device': 'cpu'}
)
print("Embedding modeli yüklendi.")

all_documents = []
print("Veri setleri okunuyor ve birleştiriliyor...")
for file_path in file_paths:
    if not os.path.exists(file_path):
        print(f"UYARI: '{file_path}' dosyası bulunamadı, atlanıyor.")
        continue
        
    with open(file_path, 'r', encoding='utf-8') as f:
        for line in f:
            try:
                item = json.loads(line)
                cleaned_question = clean_text(item['question'])
                
                page_content = f"Soru: {cleaned_question}\nCevap: {item['answer']}"
                doc = Document(page_content=page_content, metadata={"source": os.path.basename(file_path)})
                all_documents.append(doc)
            except json.JSONDecodeError:
                print(f"UYARI: '{file_path}' dosyasında hatalı bir satır atlandı.")

print(f"Toplam {len(all_documents)} adet doküman temizlendi ve birleştirildi.")

if all_documents:
    print("Birleştirilmiş verilerle ChromaDB veritabanı oluşturuluyor...")
    vectordb = Chroma.from_documents(
        documents=all_documents,
        embedding=embeddings,
        persist_directory=db_path,
        collection_name=collection_name
    )
    print(f"Veritabanı başarıyla oluşturuldu! Toplam {len(all_documents)} doküman eklendi.")
else:
    print("HATA: Veritabanına eklenecek hiçbir doküman bulunamadı. Lütfen dosya yollarını kontrol edin.")

In [ ]:

import os
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

db_path = os.path.join(project_root, "chroma_db")
collection_name = "mentormate_faq"


print("Embedding modeli ve mevcut veritabanı yükleniyor...")
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={'device': 'cpu'}
)

vectordb = Chroma(
    persist_directory=db_path, 
    embedding_function=embeddings,
    collection_name=collection_name
)

print("Veritabanı başarıyla yüklendi!")
print(f"Koleksiyondaki döküman sayısı: {vectordb._collection.count()}")

In [ ]:

from langchain.retrievers.multi_query import MultiQueryRetriever

print("Chatbot bileşenleri yükleniyor...")
chatbot_key = os.getenv("GOOGLE_API_KEY-cap") 
if not chatbot_key: raise ValueError("HATA: GOOGLE_API_KEY-cap ortam değişkeni bulunamadı.")
else: print("Gemini API anahtarı başarıyla yüklendi.")

llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash", google_api_key=chatbot_key, temperature=0.3)
db_path = os.path.join(project_root, "chroma_db")
collection_name = "mentormate_faq"
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2", model_kwargs={'device': 'cpu'})
vectordb = Chroma(persist_directory=db_path, embedding_function=embeddings, collection_name=collection_name)
print("Veritabanı başarıyla yüklendi.")


base_retriever = vectordb.as_retriever(search_kwargs={"k": 7})


retriever = MultiQueryRetriever.from_llm(
    retriever=base_retriever, llm=llm
)
print("Multi-Query Retriever başarıyla kuruldu.")

memory = ConversationBufferWindowMemory(memory_key="chat_history", k=4, return_messages=True, output_key='answer')
custom_prompt_template = """
Sen, adı MentorMate olan, iki farklı uzmanlık alanına sahip bir yapay zeka asistanısın: "Bootcamp Uzmanı" ve "Genel Konu Yardımcısı".

Görevin için izlemen gereken katı adımlar şunlardır:
1.  **Önce Dokümanları Ara:** Kullanıcının sorusunu aldığında, ilk ve tek önceliğin cevabı sana verilen DOKÜMANLAR içinde aramaktır.
2.  **Cevap Dokümanlardaysa ("Bootcamp Uzmanı" Rolü):**
    * Cevabını oluşturmak için SADECE DOKÜMANLAR'daki "Cevap:" kısmını kullan. DOKÜMANLAR'daki "Soru:" metnini asla tekrarlama.
    * Bilgiyi doğrudan, net ve profesyonel bir dille sun.
    * Asla tahmin yürütme. Güvenilirlik senin en önemli özelliğindir.
    * Cevabının sonunda mutlaka "Kaynak:" başlığı altında dokümanın 'source' bilgisini paylaş.
3.  **Cevap Dokümanlarda Yoksa ("Genel Konu Yardımcısı" Rolü):**
    * Kullanıcıya karşı tamamen dürüst ol ve durumu net bir şekilde belirt. Cevabına şu cümleyle başla: "**Bu konu bootcamp'e özel veri setimde yer almıyor. Ancak, genel bilgimle yanıtlıyorum:**"
    * Bu giriş cümlesinden sonra, soruyu genel yapay zeka bilginle en iyi şekilde yanıtla.
    * Bu modda cevap verirken kaynak gösterme.

DOKÜMANLAR:
{context}
SOHBET GEÇMİŞİ:
{chat_history}
SORU:
{question}
CEVAP:
"""
CUSTOM_PROMPT = PromptTemplate(template=custom_prompt_template, input_variables=["context", "chat_history", "question"])

chatbot_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=retriever,
    memory=memory,
    return_source_documents=True,
    combine_docs_chain_kwargs={"prompt": CUSTOM_PROMPT},
    verbose=False
)

print("="*50)
print("MentorMate Chatbot (PRO Sürüm) Hazır!")
print("="*50)
print("Sohbete başlayabilirsin...")
print("\n")

while True:
    try:
        query = input("Sen: ")
        if query.lower() in ["çıkış", "exit"]:
            print("Görüşmek üzere!")
            break
        if not query.strip(): continue
        result = chatbot_chain.invoke({"question": query})
        answer = result["answer"]
        print("\nMentorMate:")
        print(answer)
    except Exception as e:
        print(f"\nBir hata oluştu: {e}")
        break
